# Data Prepration

## how to define holiday

any day in which consecusitively there are zero sales for more than 12 hours can be treated as store closed we have to augment the data for this as given data is for the number of days and then we have converted days into hours so we can just randomly fill every 7th day as store closed and can mark all the sales to zero and we can select some random days also to flag the as zero

## how to define promotion day

sales greater than 90th percentile can be treated as promotion days we can see first the distribution of data that what are the unit sales value for 95th percentile of the data and if there are more number of days then we can push this to 97 percentile

In [1]:
import os
import pandas as pd

In [2]:
# abs_data_path = r"D:\ds_projects\kaggle\m5\data"
abs_data_path = r"C:\Users\rahulgup\Downloads\m5-forecasting-accuracy"

In [3]:
sell_price_path = os.path.join(abs_data_path, "sell_prices.csv") 
sales_data_path = os.path.join(abs_data_path, "sales_train_validation.csv") 
calendar_data_path = os.path.join(abs_data_path, "calendar.csv")

In [4]:
calendar = pd.read_csv(calendar_data_path, keep_default_na=False)
sales_train_validation = pd.read_csv(sales_data_path, keep_default_na=False)
# sample_submission = pd.read_csv("sample_submission.csv", keep_default_na=False)
# sell_prices = pd.read_csv(sell_price_path, keep_default_na=False)

In [5]:
sales_train_validation.describe()

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
count,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,...,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000
mean,1.070220,1.041292,0.780026,0.833454,0.627944,0.958052,0.918662,1.244080,1.073663,0.838701,...,1.370581,1.586159,1.693670,1.248245,1.232207,1.159167,1.149000,1.328862,1.605838,1.633158
std,5.126689,5.365468,3.667454,4.415141,3.379344,4.785947,5.059495,6.617729,5.917204,4.206199,...,3.740017,4.097191,4.359809,3.276925,3.125471,2.876026,2.950364,3.358012,4.089422,3.812248
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000
max,360.000000,436.000000,207.000000,323.000000,296.000000,314.000000,316.000000,370.000000,385.000000,353.000000,...,129.000000,160.000000,204.000000,98.000000,100.000000,88.000000,77.000000,141.000000,171.000000,130.000000


In [6]:
store = sales_train_validation.store_id.unique()[0]
store

'CA_1'

In [7]:
x = list(sales_train_validation.columns[6:])
change_col = [
    [
        "d_{}".format(x//12+1), #date_id
        x//12+1, #dt
        x%12+8 #hour
#         (x/12)%7+1 #day
    ] for x in range(len(x))
]
change_col[:5], change_col[-1:-5:-1],# change_col

([['d_1', 1, 8],
  ['d_1', 1, 9],
  ['d_1', 1, 10],
  ['d_1', 1, 11],
  ['d_1', 1, 12]],
 [['d_160', 160, 12],
  ['d_160', 160, 11],
  ['d_160', 160, 10],
  ['d_160', 160, 9]])

In [8]:
date_df = pd.DataFrame(
    change_col, columns=[
        "date_id", "dt", "hour"
        #, "day"
    ]
)

In [9]:
sales_train_validation = sales_train_validation[sales_train_validation.store_id == store]

In [10]:
sales_train = pd.melt(
    sales_train_validation,
    id_vars=sales_train_validation.columns[:6],
    value_vars=sales_train_validation.columns[6:],
    var_name='date_id',
    value_name='quantity'
).merge(
    date_df,
    on="date_id"
)

In [11]:
sales_train = sales_train[
    list(sales_train_validation.columns[:6])+list(date_df.columns)+["quantity"]
]

In [12]:
len(sales_train)

5832737

In [13]:
sales_train.head()

,id,item_id,dept_id,cat_id,store_id,state_id,date_id,dt,hour,quantity
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,1,8,0
1,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,1,9,0
2,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,1,10,0
3,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,1,11,0
4,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,1,12,0


In [14]:
calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,,,,,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,,,,,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,,,,,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,,,,,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,,,,,1,0,1


In [15]:
num_days = date_df.dt.max()
last_index = int(calendar[calendar.wday == num_days % 7].reset_index()[["index"]].max()[0])

In [16]:
last_index-num_days, last_index

(1805, 1965)

In [17]:
calendar_train = calendar[
    (calendar.index > (last_index - num_days))
    & (calendar.index <= last_index)
].reset_index()
calendar_train["date_id"] = "d_" + (calendar_train["index"] - (last_index - num_days)).map(str)
calendar_train = calendar_train.drop(
    ["index", "d", "wm_yr_wk", 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI'],
    axis=1
)

In [18]:
calendar_train.head()

,date,weekday,wday,month,year,date_id
0,2016-01-09,Saturday,1,1,2016,d_1
1,2016-01-10,Sunday,2,1,2016,d_2
2,2016-01-11,Monday,3,1,2016,d_3
3,2016-01-12,Tuesday,4,1,2016,d_4
4,2016-01-13,Wednesday,5,1,2016,d_5


In [19]:
# sales_train.to_csv(os.path.join(abs_data_path, "sales_train.csv"), index=False)

In [20]:
# calendar.to_csv(os.path.join(abs_data_path, "calendar_train.csv"), index=False)

In [ ]:
sales_train.groupby(["item_id", "store_id", "dt", "hour"]).describe()

---

# logic

## for store closure 
we can use the logic that any store showing continous 0 sales for more than we can flag them as store being closed

## to identify the run rate for the particular product 

we can run the univariate for first store product combination at row level and then at product level for the same day and  can see the results this will give us the matrix of 12 hours * 7 week days combination 

run rate will be useful measure to trigger the msg if the shelf is getting emptied 

# Core Logic

In [19]:
sales_train_final = sales_train.merge(
    calendar_train,
    on="date_id"
)

In [20]:
sales_train_final.head()

,id,item_id,dept_id,cat_id,store_id,state_id,date_id,dt,hour,quantity,date,weekday,wday,month,year
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,1,8,0,2016-01-09,Saturday,1,1,2016
1,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,1,9,0,2016-01-09,Saturday,1,1,2016
2,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,1,10,0,2016-01-09,Saturday,1,1,2016
3,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,1,11,0,2016-01-09,Saturday,1,1,2016
4,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,1,12,0,2016-01-09,Saturday,1,1,2016


In [21]:
sales_train_final.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'date_id',
       'dt', 'hour', 'quantity', 'date', 'weekday', 'wday', 'month', 'year'],
      dtype='object')

In [25]:
# sales_train_final.to_csv(os.path.join(abs_data_path, "sales_cal_train.csv"), index=False)

# Tasks
- 

In [3]:
import dask.dataframe as dd
import numpy as np
import dask.array as da

In [4]:
sell_price_new_path = os.path.join(abs_data_path, "sales_cal_train.csv") 
sales_train_final = dd.read_csv(sell_price_new_path, keep_default_na=False)

In [20]:
x = sales_train_final.groupby(
    ["store_id", "wday"]
)[["quantity"]]
y = sales_train_final.groupby(
    ["item_id", "store_id", "wday"]
)[["quantity"]]

In [21]:
# def aggr(s):
#     s['mean'] = np.mean(s)
#     s['std'] = np.std(s)
#     s['median'] = np.percentile(s, q=50)
#     s['75%'] = np.percentile(s, q=75)
#     s['85%'] = np.percentile(s, q=85)
#     s['95%'] = np.percentile(s, q=95)
#     return s

In [36]:
def aggr(s):
    s['mean'] = s.mean()
    s['std'] = s.std()
    s['median'] = s.quantile(q=.50)
    s['75%'] = s.quantile(q=.75)
    s['85%'] = s.quantile(q=.85)
    s['95%'] = s.quantile(q=.95)
    return s

In [38]:
x.apply(
    aggr
).compute()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  


ValueError: cannot reindex from a duplicate axis

In [18]:
x.apply(
    lambda x: [np.mean(x), np.std(x), np.percentile(x, q=50), np.percentile(x, q=75), np.percentile(x, q=85), np.percentile(x, q=95)]
).compute()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  


store_id  wday
CA_1      7       [[1.0022362026297742], [3.558039482521064], 0....
WI_2      1       [[0.6624552597430376], [3.1036697568492206], 0...
          6       [[0.5228224013967648], [2.4639516118591063], 0...
TX_1      2       [[0.9145122420750923], [3.701941663589874], 0....
          1       [[0.8611661699488071], [3.4687178557275757], 0...
WI_2      5       [[0.503914326864118], [2.342275819453358], 0.0...
TX_1      6       [[0.6468694105325532], [2.6851212550816026], 0...
CA_3      6       [[1.2053997837062795], [4.537048905832768], 0....
CA_2      3       [[0.686896630399133], [2.350885253639627], 0.0...
CA_1      3       [[0.9421905970596204], [3.4560520728029127], 0...
CA_2      2       [[0.987636716243387], [3.286263056552747], 0.0...
WI_3      4       [[0.9074393600182526], [3.683633505293049], 0....
CA_2      7       [[0.7904230895375532], [2.590540534759483], 0....
TX_1      5       [[0.6394969127440215], [2.6626483326650954], 0...
TX_3      4       [[0.68726738631

In [ ]:
y.apply(
    lambda x: [np.mean(x), np.std(x), np.percentile(x, q=50), np.percentile(x, q=75), np.percentile(x, q=85), np.percentile(x, q=95)]
).compute()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  


In [19]:
# x.agg(
#     [
#         "mean", "std",
# #         lambda x: np.percentile(x, q=25),
# #         lambda x: np.percentile(x, q=50),
# #         lambda x: np.percentile(x, q=75),
# #         lambda x: np.percentile(x, q=85),
# #         percentile(0.95)
#     ]
# ).compute()

In [29]:
# import dask
# A = dask.array.arange(100, chunks=3)
# df = dask.dataframe.from_dask_array(A)
# df.describe().compute()

count    100.000000
mean      49.500000
std       29.011492
min        0.000000
25%       31.500000
50%       64.500000
75%       99.000000
max       99.000000
dtype: float64

# store closure spark code

In [ ]:
# def flag_n_continuous(
#         dataframe,
#         groupings,
#         identifier,
#         threshold=2,
#         filters=None,
#         flag=1,
#         flag_name="flag",
#         second_identifier=False,
#         second_identifier_name="identifier_flag"
# ):
#     """
#     Identify N continuous rows and flag with flag value
#     """
#     # find minimum value of identifier column
#     input_columns = dataframe.columns
#     if filters:
#         dataframe = dataframe.filter(
#             filters
#         )
#     dataframe = dataframe.withColumn(
#         "min_identifier",
#         getattr(F, "min")(identifier).over(
#             W.Window.partitionBy(
#                 groupings
#             )
#         )
#     ).withColumn(
#         "diff_identifier",
#         getattr(F, "col")(identifier) - getattr(F, "col")("min_identifier") + 1
#     ).withColumn(
#         "rank",
#         getattr(F, "dense_rank")().over(
#             W.Window.partitionBy(
#                 groupings
#             ).orderBy(
#                 identifier
#             )
#         )
#     ).withColumn(
#         second_identifier_name,
#         getattr(F, "col")("diff_identifier") - getattr(F, "col")("rank")
#     ).withColumn(
#         "count",
#         getattr(F, "count")(second_identifier_name).over(
#             W.Window.partitionBy(
#                 groupings + [second_identifier_name]
#             )
#         )
#     ).withColumn(
#         flag_name,
#         F.when(
#             getattr(F, "col")("count") >= threshold,
#             F.lit(flag)
#         ).otherwise(
#             F.lit(0)
#         )
#     )
#     input_columns.append(flag_name)
#     if second_identifier:
#         input_columns.append(second_identifier_name)
#     return dataframe.select(*input_columns)


# for inventory management
